In [ ]:
# Import Libraries and Set Constant Variables


import numpy as np
from array import *
import csv
import datetime;
from bisect import bisect_left
from bisect import bisect_right

import matplotlib.pyplot as plt
import json

import multiprocessing as mp
from multiprocessing import Process, Queue
from multiprocessing import current_process
import queue
import threading
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.sql.functions import col
import pandas as pd
import random
import networkx as nx
from tqdm import tqdm
from community import modularity




In [ ]:
# Define Base and Temporary Directory Paths


BASE_ADDRESS = '/local/scratch/exported/Cardano_MCH_2023_1/'
TEMP_ADDRESS = BASE_ADDRESS + '/temp_files/'


print('----------------------')
print('done!')


----------------------
----------------------
done!


In [ ]:
# Define required methods:


##########################################################################################
def BinarySearch(a, x):
    i = bisect_left(a, x)
    if i < len(a) and a[i] == x:
        return i
    else:
        print('BinarySearch Error: -1')
        return -1

##########################################################################################
def BinarySearch_Find_start_end(a, x):
    i = bisect_left(a, x)
    j = bisect_right(a, x) - 1
    if i < len(a) and a[i] == x and j < len(a) and a[j] == x:
        return [i, j]
    else:
        print('BinarySearch Error: -1')
        print('i = ', i)
        print('j = ', j)
        return -1

##########################################################################################
def store_array_to_file_2D (input_array_name, file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Array 2D to ' + file_name + '): ', ct)

    with open(file_name, "w") as filehandle:
        json.dump(input_array_name, filehandle)
    
    et = datetime.datetime.now() - ct
    print('elapsed time (Store Array 2D to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_array_2D (file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Array 2D): ', ct)

    with open(file_name) as filehandle:
        output_array_name = json.load(filehandle)

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Array 2D): ', et)
    
    return output_array_name

##########################################################################################
def store_dict_to_file_INT (input_dict_name, file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Store Dictionary to ' + file_name + '): ', ct)

    filehandle = csv.writer(open(file_name, 'w'))
    for key, val in input_dict_name.items():
        filehandle.writerow([key, val])

    et = datetime.datetime.now() - ct
    print('elapsed time (Store Dictionary to ' + file_name + '): ', et)

    return

##########################################################################################
def load_file_to_dict_INT (file_name):
    ct = datetime.datetime.now()
    curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
    print('start time (Load ' + file_name  + ' to Dictionary): ', ct)

    filehandle = csv.reader(open(file_name, 'r'))
    output_dict_name = {int(rows[0]):int(rows[1]) for rows in filehandle}

    et = datetime.datetime.now() - ct
    print('elapsed time (Load ' + file_name  + ' to Dictionary): ', et)
    
    return output_dict_name

##########################################################################################
def BinarySearch_Find_start_end(a, x):
    i = bisect_left(a, x)
    j = bisect_right(a, x) - 1
    if i < len(a) and a[i] == x and j < len(a) and a[j] == x:
        return [i, j]
    else:
        print('BinarySearch Error: -1')
        print('i = ', i)
        print('j = ', j)
        return -1

##########################################################################################


##########################################################################################


##########################################################################################
def find_weights_graphEdges(graghEdges_array, graph_weights):
    if (len(graghEdges_array) != len(graph_weights)):
        print('find_weights_graphEdges Error: -1 (Length)')
        return -1
    
    #for i in range(len(graghEdges_array)):
    for i in tqdm(range(len(graghEdges_array))):
        nodes = np.unique(graghEdges_array[i])
        edges = graghEdges_array[i]
        edges.sort()
        
        for j in nodes:
            # (j,w)=weighted edge: j=node number, w=weight
            x = BinarySearch_Find_start_end(edges, j)
            w = x[1] - x[0] + 1
            graph_weights[i].append((i,j,w)) 
        
        #if (i % 1000000 == 0):
        #    print('One milion records done!' , i)

    return

##########################################################################################



##########################################################################################



##########################################################################################
print('----------------------')
print('done!')



----------------------
done!


In [ ]:
# Read ("sorted" "unique" array_list) [raw_address_list/payment_address_list/delegation_address_list] from file:

print('----------------------')


file_name = BASE_ADDRESS + '/Unique_AddressesListRaw__Cardano_TXs_All__2023-02-28_143357.txt'
unique_raw_addresses = load_file_to_array (file_name)
print('Length of \"unique_raw_addresses\" = ' + str(len(unique_raw_addresses)))


file_name = BASE_ADDRESS + '/Unique_AddressesListPayment__Cardano_TXs_All__2023-02-28_143953.txt'
unique_payment_addresses = load_file_to_array (file_name)
print('Length of \"unique_payment_addresses\" = ' + str(len(unique_payment_addresses)))


file_name = BASE_ADDRESS + '/Unique_AddressesListDelegation__Cardano_TXs_All__2023-02-28_144415.txt'
unique_delegation_addresses = load_file_to_array (file_name)
print('Length of \"unique_delegation_addresses\" = ' + str(len(unique_delegation_addresses)))



##########################################################################################
INITIAL_DATE_CARDANO      = datetime.datetime.strptime('2017-09-23 21:44:51', '%Y-%m-%d %H:%M:%S').date()
FINAL_DATE_CARDANO        = datetime.datetime.strptime('2023-01-21 17:39:30', '%Y-%m-%d %H:%M:%S').date()
total_time_length_CARDANO = int((FINAL_DATE_CARDANO - INITIAL_DATE_CARDANO).total_seconds()/86400) + 1

unique_raw_addresses_len        = len(unique_raw_addresses)
unique_payment_addresses_len    = len(unique_payment_addresses)
unique_delegation_addresses_len = len(unique_delegation_addresses)


##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# Read Address Clustering Results from Files:

file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC__Cardano_TXs_All__2023-02-25_223957.txt'
clustering_array_heur1 = load_file_to_array (file_name)


file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic2__Cardano_TXs_All__2023-03-26_110150.txt'
clustering_array_heur2 = load_file_to_array (file_name)


file_name = BASE_ADDRESS + '/clusteringArrayList_Heuristic1noSC_AND_Heuristic2__Cardano_TXs_All__2023-03-26_141212.txt'
clustering_array_heur1and2 = load_file_to_array (file_name)


##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# Load graphEdges_merged from file: 



print('----------------------')
# it contains all edges of the address network graph.
# "graphEdges_merged" contains all identified edges in all transactions.:


graphEdges_merged = load_file_to_array_2D(BASE_ADDRESS + '/graphEdgesArrayList_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-25_224222.txt')

print('Lenght of \"graphEdges_merged\" = ', len(graphEdges_merged))

##########################################################################################
print('----------------------')
print('done!')


In [ ]:
# Calculate graph_weights (everytime that two addresses are identified to be clustered together according to 
# a heuristic in a transaction, the weight of the edge between the two addresses is increased by 1):

print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


##########################################################################################
graph_weights = [[] for _ in range(unique_addresses_len)]
find_weights_graphEdges(graphEdges_merged, graph_weights)

##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Calculate graph_weights): ", et)


##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# Store/Load "graph_weights" into/from file:



# Store graph_weights into file:

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


ct = datetime.datetime.now()
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/graphWeightsArrayList_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__' + curr_timestamp + '.txt'
print('output_filename = ', output_filename)

with open(output_filename, 'w') as filehandle:
        for element in graph_weights:
            filehandle.write(f'{element}\n')


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Store graph_weights into file): ", et)

##########################################################################################
print('----------------------')
print('done!')




# Load graph_weights from file:

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


input_filename = BASE_ADDRESS + '/graphWeightsArrayList_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-25_234559.txt'
print('input_filename = ', input_filename)


graph_weights = [[] for _ in range(unique_addresses_len)]

i=0
with open(input_filename) as filehandle:
    for row in filehandle:
        graph_weights[i] = ast.literal_eval(row[:-1])
        i = i+1
        if (i % 1000000 == 0):
            print('One milion records done!' , i)


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Store graph_weights into file): ", et)

##########################################################################################
print('----------------------')
print('done!')





In [ ]:
# Generate "Graph G Addrs network" using graph_weights:

print('----------------------')

# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


G = nx.Graph()

for i in tqdm(range(len(graph_weights))):
    G.add_node(i)
    G.add_weighted_edges_from(graph_weights[i])


print('----------------------')
print('Is Connected    (G) = ', nx.is_connected(G))
print('Number of Nodes (G) = ', G.number_of_nodes())
print('Number of Edges (G) = ', G.number_of_edges())


print('----------------------')
test_results = [39057080, 454231, 397965]
for i in test_results:
    print('----------------------')
    print('Degree of node ' + str(i) + ' = ', G.degree()[i])
    print('graph_weights[' + str(i) + '] = ', graph_weights[i])


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Store graph_weights into file): ", et)


##########################################################################################
print('----------------------')
print('done!')


In [ ]:
# Store/Load graph object "Graph G Addrs network" to/from file:


# Store graph object to file:

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


import pickle
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/Graph_G_AddrsNetwork_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__' + curr_timestamp + '.pickle'
print('output_filename = ', output_filename)
pickle.dump(G, open(output_filename, 'wb'))


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Store G into file): ", et)

##########################################################################################
print('----------------------')
print('done!')




# Load graph object from file:

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


import pickle
input_filename = BASE_ADDRESS + '/Graph_G_AddrsNetwork_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_000507.pickle'
print('input_filename = ', input_filename)
G = pickle.load(open(input_filename, 'rb'))


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Load G from file): ", et)

##########################################################################################
print('----------------------')
print('done!')




In [ ]:
# Extract the largest connected component:

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)



#largest_cc = max(nx.connected_components(G), key=len)
largest_cc = list(sorted(nx.connected_components(G), key=len, reverse=True))[0]
largest_cc_subgraph = G.subgraph(largest_cc).copy()

print('Is Connected    (largest_cc_subgraph) = ', nx.is_connected(largest_cc_subgraph))
print('Number of Nodes (largest_cc_subgraph) = ', largest_cc_subgraph.number_of_nodes())
print('Number of Edges (largest_cc_subgraph) = ', largest_cc_subgraph.number_of_edges())



print('----------------------')
# Find weighted degrees of nodes:
degree_sequence = sorted((d for n, d in largest_cc_subgraph.degree(weight='weight')), reverse=False)
print('Sum Weighted Degrees (largest_cc_subgraph) = ', sum(degree_sequence))
print('Max Weighted Degrees (largest_cc_subgraph) = ', max(degree_sequence))


print('----------------------')
for i in range(1, 15):
    x = BinarySearch_Find_start_end(degree_sequence, i)
    print('Number of nodes with Weighted Degree \"' + str(i) + '\" = ', x[1] - x[0] + 1)

##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Store graph_weights into file): ", et)

##########################################################################################
print('----------------------')
print('done!')


In [ ]:
# Store/Load graph object "largest_cc_subgraph" to/from file:


# Store graph object to file:

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


import pickle
curr_timestamp = str(ct)[0:10] + '_' + str(ct)[11:13] + str(ct)[14:16] + str(ct)[17:19]
output_filename = BASE_ADDRESS + '/Largest1_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__' + curr_timestamp + '.pickle'
print('output_filename = ', output_filename)
pickle.dump(largest_cc_subgraph, open(output_filename, 'wb'))


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Store largest_cc_subgraph into file): ", et)

##########################################################################################
print('----------------------')
print('done!')




# Load graph object from file:

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


import pickle
input_filename = BASE_ADDRESS + '/Largest2_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-01-10_181953.pickle'
print('input_filename = ', input_filename)
largest_cc_subgraph = pickle.load(open(input_filename, 'rb'))


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Load largest_cc_subgraph into file): ", et)

##########################################################################################
print('----------------------')
print('done!')




In [ ]:
# Load 9 superclusters (largest_cc_subgraphs):

import pickle


print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


print('----------------------')
input_filename = BASE_ADDRESS + '/Largest1_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_140703.pickle'
largest_CCs_1 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_1 loaded!')

input_filename = BASE_ADDRESS + '/Largest2_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_080139.pickle'
largest_CCs_2 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_2 loaded!')

input_filename = BASE_ADDRESS + '/Largest3_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_083614.pickle'
largest_CCs_3 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_3 loaded!')

input_filename = BASE_ADDRESS + '/Largest4_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_085210.pickle'
largest_CCs_4 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_4 loaded!')

input_filename = BASE_ADDRESS + '/Largest5_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_085920.pickle'
largest_CCs_5 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_5 loaded!')

input_filename = BASE_ADDRESS + '/Largest6_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_085927.pickle'
largest_CCs_6 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_6 loaded!')

input_filename = BASE_ADDRESS + '/Largest7_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_085927.pickle'
largest_CCs_7 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_7 loaded!')

input_filename = BASE_ADDRESS + '/Largest8_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_085927.pickle'
largest_CCs_8 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_8 loaded!')

input_filename = BASE_ADDRESS + '/Largest9_cc_subgraph_Heuristic1noSC_LinkToALLAddressesInTX__Cardano_TXs_All__2023-02-26_085927.pickle'
largest_CCs_9 = pickle.load(open(input_filename, 'rb'))
print('largest_CCs_9 loaded!')



##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time: ", et)

##########################################################################################
print('----------------------')
print('done!')


In [ ]:
# PLOT: degree distribution of 9 superclusters:

plt.style.use('https://raw.githubusercontent.com/benckj/mpl_style/main/uzh.mplstyle')

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


print('----------------------')
# Find NoWeight degrees of nodes:
noWeight_degree_seq_1 = sorted((d for n, d in tqdm(largest_CCs_1.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_1, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_1)),40))

noWeight_degree_seq_2 = sorted((d for n, d in tqdm(largest_CCs_2.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_2, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_2)),40))

noWeight_degree_seq_3 = sorted((d for n, d in tqdm(largest_CCs_3.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_3, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_3)),40))

noWeight_degree_seq_4 = sorted((d for n, d in tqdm(largest_CCs_4.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_4, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_4)),40))

noWeight_degree_seq_5 = sorted((d for n, d in tqdm(largest_CCs_5.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_5, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_5)),40))

noWeight_degree_seq_6 = sorted((d for n, d in tqdm(largest_CCs_6.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_6, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_6)),40))

noWeight_degree_seq_7 = sorted((d for n, d in tqdm(largest_CCs_7.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_7, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_7)),40))

noWeight_degree_seq_8 = sorted((d for n, d in tqdm(largest_CCs_8.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_8, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_8)),40))

noWeight_degree_seq_9 = sorted((d for n, d in tqdm(largest_CCs_9.degree())), reverse=True) # sort by NodeDegree_NoWeight
plt.hist(noWeight_degree_seq_9, density=True, histtype='step', bins = np.logspace(np.log10(1), np.log10(max(noWeight_degree_seq_9)),40))


plt.xlabel('Degree')
plt.ylabel('Probability')

plt.legend(['Supercluster 1', 'Supercluster 2', 'Supercluster 3', 'Supercluster 4', 'Supercluster 5', 'Supercluster 6', 'Supercluster 7', 'Supercluster 8', 'Supercluster 9'], fontsize="8", loc ="upper right")

plt.xscale("log" )
plt.yscale("log" )

plt.savefig('fig_deg_dist_super_clusters.pdf', bbox_inches='tight', facecolor='white')
plt.show()



##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time: ", et)

##########################################################################################
print('----------------------')
print('done!')



In [ ]:
# PLOT: power-law distributions (Superclusters degree distribution):


fit_1 = powerlaw.Fit(noWeight_degree_seq_1)
print('fit.power_law.alpha = ', fit_1.power_law.alpha)
print('fit.power_law.sigma = ', fit_1.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_1.distribution_compare('power_law', 'exponential'))

fit_2 = powerlaw.Fit(noWeight_degree_seq_2)
print('fit.power_law.alpha = ', fit_2.power_law.alpha)
print('fit.power_law.sigma = ', fit_2.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_2.distribution_compare('power_law', 'exponential'))

fit_3 = powerlaw.Fit(noWeight_degree_seq_3)
print('fit.power_law.alpha = ', fit_3.power_law.alpha)
print('fit.power_law.sigma = ', fit_3.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_3.distribution_compare('power_law', 'exponential'))

fit_4 = powerlaw.Fit(noWeight_degree_seq_4)
print('fit.power_law.alpha = ', fit_4.power_law.alpha)
print('fit.power_law.sigma = ', fit_4.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_4.distribution_compare('power_law', 'exponential'))

fit_5 = powerlaw.Fit(noWeight_degree_seq_5)
print('fit.power_law.alpha = ', fit_5.power_law.alpha)
print('fit.power_law.sigma = ', fit_5.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_5.distribution_compare('power_law', 'exponential'))

fit_6 = powerlaw.Fit(noWeight_degree_seq_6)
print('fit.power_law.alpha = ', fit_6.power_law.alpha)
print('fit.power_law.sigma = ', fit_6.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_6.distribution_compare('power_law', 'exponential'))

fit_7 = powerlaw.Fit(noWeight_degree_seq_7)
print('fit.power_law.alpha = ', fit_7.power_law.alpha)
print('fit.power_law.sigma = ', fit_7.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_7.distribution_compare('power_law', 'exponential'))

fit_8 = powerlaw.Fit(noWeight_degree_seq_8)
print('fit.power_law.alpha = ', fit_8.power_law.alpha)
print('fit.power_law.sigma = ', fit_8.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_8.distribution_compare('power_law', 'exponential'))

fit_9 = powerlaw.Fit(noWeight_degree_seq_9)
print('fit.power_law.alpha = ', fit_9.power_law.alpha)
print('fit.power_law.sigma = ', fit_9.power_law.sigma)
print('fit.distribution_compare(\'power_law\', \'exponential\') = ', fit_9.distribution_compare('power_law', 'exponential'))


###############################################################################################################################################
plt.style.use('https://raw.githubusercontent.com/benckj/mpl_style/main/uzh.mplstyle')
plt.xlabel('Degree')
plt.ylabel('Probability Density')

plt.ylim(bottom=1e-12)

###############################################################################################################################################
markersize = 5

fig = fit_1.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_1.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_2.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_2.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_3.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_3.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_4.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_4.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_5.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_5.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_6.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_6.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_7.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_7.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_8.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_8.power_law.plot_pdf(linestyle = 'dashed', ax = fig)

fig = fit_9.plot_pdf(linestyle = '', marker='o', original_data=True, markersize=markersize)
#fit_9.power_law.plot_pdf(linestyle = 'dashed', ax = fig)


plt.legend(['Supercluster 1', 'Supercluster 2', 'Supercluster 3', 'Supercluster 4', 'Supercluster 5', 'Supercluster 6', 'Supercluster 7', 'Supercluster 8', 'Supercluster 9'], fontsize="10", loc ="upper right")

plt.savefig('fig_deg_dist_superclusters.pdf', bbox_inches='tight', facecolor='white')



In [ ]:
# Perform "Label Propagation" on the largest_cc_subgraph_COPY (largest cluster):

print('----------------------')
# ct stores current time
ct = datetime.datetime.now()
print("current time: ", ct)


if __name__ == "__main__":
    """
    --rounds      INT    Number of iterations    Default is 8.
    --seed        INT    Initial seed            Default is 42.
    """
    args = {'rounds': 8, 'seed': 42}
    model = LabelPropagator(largest_cc_subgraph_COPY, args)
    NodeLabelsDict = model.do_a_series_of_propagations()
    #NodeLabelsDict = dict(sorted(NodeLabelsDict.items()))                          # sorted by key
    NodeLabelsDict = dict(sorted(NodeLabelsDict.items(), key=lambda item: item[1])) # sorted by value


##########################################################################################
print('----------------------')
et = datetime.datetime.now() - ct
print("Total elapsed time (Store graph_weights into file): ", et)

##########################################################################################
print('----------------------')
print('done!')

